In [2]:
import json
import pandas as pd
from dash import Dash, html, dcc, callback, Input, Output
import plotly.graph_objs as go

# ─── 1. Load data ─────────────────────────────────────
with open('data/production.json') as f:
    prod = pd.DataFrame(json.load(f))
prod['timestamp'] = pd.to_datetime(prod['timestamp'])

with open('data/commercial.json') as f:
    comm = pd.DataFrame(json.load(f))
comm['timestamp'] = pd.to_datetime(comm['timestamp'])

reg = pd.DataFrame(json.load(open('data/regulatory.json')))

# merge prod & comm on timestamp & site_id
df = prod.merge(comm, on=['timestamp','site_id'])

# current month
today = pd.Timestamp.today().normalize()
mtd_mask = df['timestamp'].dt.to_period('M') == today.to_period('M')
mtd = df[mtd_mask].copy()

# MTD metrics
mtd_energy  = mtd['energy_mwh'].sum()
mtd_revenue = mtd['revenue_usd'].sum()

# regulatory target for this month
month_str = today.to_period('M').strftime('%Y-%m')
target_row = reg[reg['month']==month_str].iloc[0]
eqr, ferc, eia = target_row[['eqr_mwh','ferc_mwh','eia_mwh']]

# variance & compliance
def pct(var, actual): return (actual - var) / var * 100
var_eqr  = pct(eqr,  mtd_energy)
compliant_eqr = '✓' if abs(var_eqr) <= 5 else '⚠'

# ─── 2. Build Figures ─────────────────────────────────

# A) Line chart: energy actual vs EQR target, daily
daily = df.resample('D', on='timestamp').agg({'energy_mwh':'sum'}).reset_index()
daily['eqr'] = eqr / today.day * daily['timestamp'].dt.day  # straight‐line budget
line_fig = go.Figure()
line_fig.add_trace(go.Scatter(x=daily['timestamp'], y=daily['energy_mwh'],
                              mode='lines+markers', name='Actual MWh'))
line_fig.add_trace(go.Scatter(x=daily['timestamp'], y=daily['eqr'],
                              mode='lines', name='EQR Budget',
                              line=dict(dash='dash')))
line_fig.update_layout(yaxis_title='MWh', xaxis_title='', template='plotly_white')

# B) Bar chart: actual vs FERC & EIA for last 12 months
reg['month_dt'] = pd.to_datetime(reg['month'] + '-01')
bar_fig = go.Figure()
bar_fig.add_trace(go.Bar(x=reg['month_dt'], y=reg['eqr_mwh'], name='EQR'))
bar_fig.add_trace(go.Bar(x=reg['month_dt'], y=reg['ferc_mwh'], name='FERC'))
bar_fig.add_trace(go.Bar(x=reg['month_dt'], y=reg['eia_mwh'],  name='EIA'))
bar_fig.update_layout(barmode='group',
                      yaxis_title='MWh', xaxis_title='Month',
                      template='plotly_white')

# ─── 3. Assemble Dash layout ─────────────────────────────────

app = Dash(__name__, title='Regulatory Reporting Dashboard')

app.layout = html.Div(style={'fontFamily':'Arial, sans-serif','margin':'40px'}, children=[
    html.H1('Regulatory Reporting Dashboard', style={'textAlign':'center'}),

    # KPI Cards
    html.Div(style={'display':'flex','justifyContent':'space-around','marginBottom':'40px'}, children=[
        html.Div(style={'padding':'20px','border':'1px solid #ddd','borderRadius':'10px','width':'18%','textAlign':'center'}, children=[
            html.H4('MTD Production'),
            html.H2(f"{mtd_energy:,.0f} MWh")
        ]),
        html.Div(style={'padding':'20px','border':'1px solid #ddd','borderRadius':'10px','width':'18%','textAlign':'center'}, children=[
            html.H4('MTD Revenue'),
            html.H2(f"${mtd_revenue:,.0f}")
        ]),
        html.Div(style={'padding':'20px','border':'1px solid #ddd','borderRadius':'10px','width':'18%','textAlign':'center'}, children=[
            html.H4('EQR Variance'),
            html.H2(f"{var_eqr:+.1f}% {compliant_eqr}")
        ])
    ]),

    # Charts
    html.Div([
        dcc.Graph(figure=line_fig, config={'displayModeBar':False}),
        dcc.Graph(figure=bar_fig, config={'displayModeBar':False})
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\bschm\AppData\Local\Temp\ipykernel_26796\799975631.py:22: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  mtd_mask = df['timestamp'].dt.to_period('M') == today.to_period('M')


IndexError: single positional indexer is out-of-bounds